In [ ]:
import astropy.units as u
import numpy as np
from astroquery.gaia import Gaia
from joblib import Memory
import matplotlib.pyplot as plt
from matplotlib import colors

In [ ]:
def get_gaia_query_results(ra=66.75, dec=15.86, radius=2, conds="", limit=50000):
    add = ""
    if conds != "":
        add = f"AND {conds}"
    query = f"""
    SELECT TOP {limit} *
    FROM gaiaedr3.gaia_source
    WHERE 
        CONTAINS(
            POINT('ICRS',gaiaedr3.gaia_source.ra,gaiaedr3.gaia_source.dec),
            CIRCLE('ICRS',{ra},{dec},{radius})
        )=1
    """ + add
    job = Gaia.launch_job_async(query)
    return job.get_results()

In [ ]:
location = "./cachedir"
memory = Memory(location, verbose=0)
get_gaia_query_results_cached = memory.cache(get_gaia_query_results)

In [ ]:
stringent_conds = '''
    parallax_over_error > 10
    AND ABS(parallax - 21.052) < 5
    AND ABS(pmra-4.614) < 300
    AND ABS(pmdec+7.705) < 400
    AND phot_g_mean_flux_over_error>25
    AND phot_rp_mean_flux_over_error>10
    AND phot_bp_mean_flux_over_error>10
    AND pmra_error < 0.1
    AND pmdec_error < 0.1
    AND phot_bp_rp_excess_factor < 1.3+0.06*power(phot_bp_mean_mag-phot_rp_mean_mag,2)
    AND phot_bp_rp_excess_factor > 1.0+0.015*power(phot_bp_mean_mag-phot_rp_mean_mag,2)
    AND astrometric_excess_noise < 1
'''
print("Starting stringent query...")
res_stringent = get_gaia_query_results_cached(radius = 5, conds = stringent_conds)

In [ ]:
less_conds = '''
    parallax_over_error > 10
    AND phot_g_mean_flux_over_error>50
    AND phot_rp_mean_flux_over_error>20
    AND phot_bp_mean_flux_over_error>20
    AND pmra_error < 0.1
    AND pmdec_error < 0.1
    AND phot_bp_rp_excess_factor < 1.3+0.06*power(phot_bp_mean_mag-phot_rp_mean_mag,2)
    AND phot_bp_rp_excess_factor > 1.0+0.015*power(phot_bp_mean_mag-phot_rp_mean_mag,2)
    AND astrometric_excess_noise < 1
'''
print("Starting condless query...")
res_condless = get_gaia_query_results_cached(radius = 5, conds=less_conds)

### Stringent Conditions

In [ ]:
res_stringent

In [ ]:
bp_rp = res_stringent['bp_rp'].data
phot_g_mean_mag = res_stringent['phot_g_mean_mag'].data
parallax = res_stringent['parallax'].data
ra = res_stringent['pmra'].data
dec = res_stringent['pmdec'].data
mg = phot_g_mean_mag+5*np.log10(parallax)-10

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
h = ax.hist2d(bp_rp,mg,bins=300, norm=colors.PowerNorm(0.5), zorder=0.5)
ax.scatter(bp_rp, mg, alpha=0.05, s=1, color='k', zorder=0)
ax.invert_yaxis()
cb = plt.colorbar(h[3], ax=ax, pad=0.02)
plt.show()

In [ ]:
print(f"RA mean: {np.mean(ra)}, DEC mean:{np.mean(dec)}")
print(f"RA max: {np.max(ra)}, DEC max: {np.max(dec)}")
print(f"RA min: {np.min(ra)}, DEC min: {np.min(dec)}")

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
ax.scatter(ra, dec, s=1, color='k')
plt.xlabel("Proper Motion: Right Ascension")
plt.ylabel("Proper Motion: Declination")
plt.show()

### Conditionless (Limit 50,000)


In [ ]:
res_condless

In [ ]:
res_condless_pd = res_condless.to_pandas()
res_condless_pd = res_condless_pd[res_condless_pd['parallax'].notna()]
res_condless_pd = res_condless_pd[res_condless_pd['parallax'] > 0.5]
res_condless_pd = res_condless_pd[res_condless_pd['bp_rp'].notna()]

In [ ]:
bp_rp = res_condless_pd['bp_rp']
phot_g_mean_mag = res_condless_pd['phot_g_mean_mag']
parallax = res_condless_pd['parallax']
ra = res_condless_pd['pmra']
dec = res_condless_pd['pmdec']
mg = phot_g_mean_mag+5*np.log10(parallax)-10

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
h = ax.hist2d(bp_rp,mg,bins=300, norm=colors.PowerNorm(0.5), zorder=0.5)
ax.scatter(bp_rp, mg, alpha=0.05, s=1, color='k', zorder=0)
ax.invert_yaxis()
cb = plt.colorbar(h[3], ax=ax, pad=0.02)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
ax.scatter(ra, dec, alpha=0.05, s=1, color='k', zorder=0)
ax.invert_yaxis()
cb = plt.colorbar(h[3], ax=ax, pad=0.02)
plt.show()

## Isochrones, MIST Models

In [ ]:
import sys
sys.path.append(".")
from read_mist_models import ISOCMD

In [ ]:
mist_fname = 'MIST_iso_6017590e63448.iso.cmd'
isocmd = ISOCMD(mist_fname)

In [ ]:
print('version: ', isocmd.version)
print('photometric system: ', isocmd.photo_sys)
print('abundances: ', isocmd.abun)
print('rotation: ', isocmd.rot)
print('ages: ', [round(x,2) for x in isocmd.ages])
print('number of ages: ', isocmd.num_ages)
print('available columns: ', isocmd.hdr_list)
print('Av extinction: ', isocmd.Av_extinction)

In [ ]:
def plot_temp_lum(isocmd = isocmd, age_index=5.0):
    age_ind = isocmd.age_index(age_index)
#     logTeff = isocmd.isocmds[age_ind]['log_Teff']
    logTeff = isocmd.isocmds[age_ind]['Gaia_BP_DR2Rev'] - isocmd.isocmds[age_ind]['Gaia_RP_DR2Rev']
    logL = isocmd.isocmds[age_ind]['log_L']
#     print(isocmd.isocmds[age_ind]['[Fe/H]'])
    plt.plot(logTeff, logL)
    plt.xlabel("logTeff")
    plt.ylabel("logL")

In [ ]:
ages = np.array(isocmd.ages)
for age in ages[::5]:
    plot_temp_lum(isocmd, age_index = age)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
h = ax.hist2d(bp_rp,mg,bins=300, norm=colors.PowerNorm(0.5), zorder=0.5)
ax.scatter(bp_rp, mg, alpha=0.05, s=1, color='k', zorder=0)
plot_temp_lum(isocmd_2, 8.9, invert_y = False)
ax.invert_yaxis()
cb = plt.colorbar(h[3], ax=ax, pad=0.02)
plt.show()